### *IT3052E - Fundamentals of Optimization*
# **Mini Project 18 - Nurse Scheduling Problem**
#### **Techniques used**:
* Backtracking,
* Constraint Programming,
* Linear Programming,
* Local Search, and
* Meta-heuristics (Genetic Algorithm).

#### * ***Import pandas for printing solution***

In [124]:
import pandas as pd
import random
import time

## 3. Hill-climbing search

### 3.1. Read data from files

In [125]:
with open('SampleData/testCase2/4.txt') as file:
  N, D, a, b = [int(q) for q in file.readline().split()]
  dayoff = [[0 for d in range(D)] for n in range(N)]
  for n in range(N):
    for d in [int(h) for h in file.readline().split()]:
      if d != -1:
            dayoff[n][d-1] = 1
            
starttime = time.time()

### 3.2. Initialize the solution

In [126]:
def initialize(nurses, D, a, b):
    # Initialize the schedule randomly, but ensuring each nurse works one shift per day
    # and accounting for the day off constraint
    schedule = [[None for _ in range(D)] for _ in range(nurses)]
    for day in range(D):
        available_nurses = [i for i in range(nurses) if schedule[i][day] is None]
        for shift in range(4):
            assigned_nurses = set()
            while len(assigned_nurses) < a:
                if not available_nurses:
                    # If there aren't enough nurses available for this shift, reinitialize the schedule
                    return initialize(nurses, D)
                
                nurse = random.choice(available_nurses)
                if day > 0 and schedule[nurse][day-1] == 3:
                    # Nurse worked evening shift yesterday, so they have today off
                    available_nurses.remove(nurse)
                    continue
                
                available_nurses.remove(nurse)
                assigned_nurses.add(nurse)
                schedule[nurse][day] = shift
                
                if len(assigned_nurses) == b:
                    break
    return schedule

### 3.3. Evaluate the objective function

In [127]:
def evaluate(schedule):
    # Calculate the maximum number of night shifts assigned to any nurse
    night_shift_counts = [sum(1 for shift in nurse_schedule if shift == 3) for nurse_schedule in schedule]
    return max(night_shift_counts)

### 3.4. Hill-climbing main algorithm

#### 3.4.1. Generate neighbors

In [128]:
def generate_neighbors(schedule, D):
    # Generate all possible neighbors by swapping the shift of two nurses on a single day
    neighbors = []
    for day in range(D):
        for nurse1 in range(len(schedule)):
            for nurse2 in range(nurse1+1, len(schedule)):
                neighbor = [row[:] for row in schedule]
                neighbor[nurse1][day], neighbor[nurse2][day] = neighbor[nurse2][day], neighbor[nurse1][day]
                neighbors.append(neighbor)
    return neighbors

#### 3.4.2. Hill climbing algorithm

In [129]:
def hill_climbing(nurses, D, a, b):
    # Generate an initial solution
    current_state = initialize(nurses, D, a, b)
    
    while True:
        # Generate all possible neighbors of the current solution
        neighbors = generate_neighbors(current_state, D)
        
        # Evaluate the neighbors and find the best one
        best_neighbor = min(neighbors, key=evaluate)
        best_neighbor_value = evaluate(best_neighbor)
        
        # If the best neighbor is better than the current state, move to the best neighbor
        if best_neighbor_value < evaluate(current_state):
            current_state = best_neighbor
        # Otherwise, return the current state
        else:
            return current_state

### 3.6. Print the schedule

In [ ]:
schedule = hill_climbing(N, D, a, b)
res = [[0 for n in range(N)] for d in range(D)]
for i, nurse_schedule in enumerate(schedule):
    #print(f"Nurse {i+1}: ", end="")
    j = 0
    for shift in nurse_schedule:
        if shift is None:
            res[j][i] = 0
        else:
            res[j][i] = shift+1
        j += 1
df = pd.DataFrame(res, index = [d+1 for d in range(D)], columns = [n+1 for n in range(N)])
df.index.name = 'Day'
df.columns.name = 'Nurse'
#display(df)
print('Running time:',time.time() - starttime)

### 3.7. Optimal solution

In [ ]:
print('Optimal solution - Max night shift assigned to a nurse:', evaluate(schedule))